In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os

2023-12-24 10:53:24.577893: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-24 10:53:25.524749: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def checkfilepath(filepath):
    return os.path.exists(filepath)

def read_nifti_file(filepath):
    scan = nib.load(filepath)
    data = scan.get_fdata()
    return data

def normalize(volume):
    dmin = np.amin(volume)
    dmax = np.amax(volume)
    davg = np.average(volume)
    volume = (volume-dmin)/davg
    return volume

def process_scan(path):
    vol = read_nifti_file(path)
    vol = normalize(vol)
    return vol


In [3]:
df = pd.read_excel('/lfs1/ashaji/Imputation_Problem/data/ADNI_final.xlsx', engine='openpyxl')
df['SubjID'].replace('',np.nan,inplace=True)
df['AGE_at_scan'].replace('',np.nan,inplace=True)
df.dropna(subset=['SubjID','AGE_at_scan'], inplace=True)
df = df.reset_index(drop=True)
df.loc[df['SEX']=='M','SEX'] = 1
df.loc[df['SEX']=='F','SEX'] = 0
condCN = df['DX'] == 'CN'
df = df.sort_values(by = ['SubjID'])
df = df.loc[condCN,:]
# df = df.sample(frac = 1)
df = df.reset_index(drop=True)

In [4]:
df['SEX'].replace('',np.nan,inplace=True)
df['CDRSB'].replace('',np.nan,inplace=True)
df['ADAS11'].replace('',np.nan,inplace=True)
df['ADAS13'].replace('',np.nan,inplace=True)
df['MMSE'].replace('',np.nan,inplace=True)
df['MOCA'].replace('',np.nan,inplace=True)
df['APOE A1'].replace('',np.nan,inplace=True)
df['APOE A2'].replace('',np.nan,inplace=True)
# df.dropna(subset=['SEX','CDRSB','ADAS11','ADAS13','MMSE','MOCA','APOE A1','APOE A2'], inplace=True)

In [5]:
cond = [checkfilepath(str(x)) for x in df['ACCEL_DL_6DOF_2MM_T1']]
cond0 = df['ACCEL_Preprocessed for DL?']=='yes'

In [6]:
conddwi = [checkfilepath(str(x)) for x in df['DWI_Matched_File_FA_Path_ENIGMATBSSspace']]

In [7]:
cond2 = df['SEX'].isnull()
cond2 = np.invert(cond2)
cond3 = df['CDRSB'].isnull()
cond3 = np.invert(cond3)
cond4 = df['ADAS11'].isnull()
cond4 = np.invert(cond4)
cond5 = df['ADAS13'].isnull()
cond5 = np.invert(cond5)
cond6 = df['MMSE'].isnull()
cond6 = np.invert(cond6)
cond7 = df['MOCA'].isnull()
cond7 = np.invert(cond7)
cond8 = df['APOE A1'].isnull()
cond8 = np.invert(cond8)
cond9 = df['APOE A2'].isnull()
cond9 = np.invert(cond9)

In [8]:
condt1 = [cond[i] & cond0[i]  for i in range(len(cond))]

In [9]:
condstr = cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8 & cond9

In [10]:
t1dwistr = df.loc[[(condt1[i] and conddwi[i] and condstr[i]) for i in range(len(condt1))]  , :]
len(t1dwistr)

458

In [11]:
t1dwi_str = df.loc[[(condt1[i] and not conddwi[i] and condstr[i]) for i in range(len(condt1))] , :]
len(t1dwi_str)

1255

In [12]:
t1dwi_str_ = df.loc[[(condt1[i] and not conddwi[i] and not condstr[i]) for i in range(len(condt1))] , :]
len(t1dwi_str_)

329

In [13]:
t1_dwi_str = df.loc[[(not condt1[i] and not conddwi[i] and condstr[i]) for i in range(len(condt1))] , :]
len(t1_dwi_str)

107

In [14]:
t1dwistr_ = df.loc[[(condt1[i] and conddwi[i] and not condstr[i]) for i in range(len(condt1))] , :]
len(t1dwistr_)

74

In [15]:
t1_dwistr_ = df.loc[[(not condt1[i] and conddwi[i] and not condstr[i]) for i in range(len(condt1))] , :]
len(t1_dwistr_)

0

In [16]:
t1_dwistr = df.loc[[(not condt1[i] and conddwi[i] and condstr[i]) for i in range(len(condt1))] , :]
len(t1_dwistr)

13

In [17]:
t1dwi_str_ = df.loc[[(condt1[i] and not conddwi[i] and not condstr[i]) for i in range(len(condt1))] , :]
len(t1dwi_str_)

329

In [18]:
df = df.reset_index(drop=True)
df.head(20)

,SubjID,RID,NONACCEL_T1_SCAN_FILENAME,NONACCEL_T1_IMAGEID,NONACCEL_Path to RAW nifti,NONACCEL_Preprocessed for DL?,NONACCEL_DL_6DOF_2MM_T1,NONACCEL_DL_6DOF_2MM_MASK,NONACCEL_DL_6DOF_2MM_GM,NONACCEL_DL_6DOF_2MM_WM,...,DWI_Matched_File_L1_Path_ENIGMATBSSspace,DWI_Matched_File_MD_Path_ENIGMATBSSspace,DWI_Matched_File_RD_Path_ENIGMATBSSspace,T1_Path_ENIGMATBSSspace,DWI_Matched_File_FA_Path_ENIGMATBSSspace_2MM,DWI_Matched_File_L1_Path_ENIGMATBSSspace_2MM,DWI_Matched_File_MD_Path_ENIGMATBSSspace_2MM,DWI_Matched_File_RD_Path_ENIGMATBSSspace_2MM,T1_Path_ENIGMATBSSspace_2MM,DWI Preprocessing Pipeline (old=ADNI2 - new=ADNI3)
0,002_S_0295,295,002_S_0295_20060418_A1_T1_1.5T_nonaccel_Preproc,45108.0,/nas/bioint-data/neuroimaging-data/ADNI/Organi...,yes,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,002_S_0295,295,002_S_0295_20061102_A1_T1_1.5T_nonaccel_Preproc,40966.0,/nas/bioint-data/neuroimaging-data/ADNI/Organi...,yes,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,002_S_0295,295,002_S_0295_20070525_A1_T1_1.5T_nonaccel_Preproc,64025.0,/nas/bioint-data/neuroimaging-data/ADNI/Organi...,yes,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,002_S_0295,295,002_S_0295_20080723_A1_T1_1.5T_nonaccel_Preproc,123685.0,/nas/bioint-data/neuroimaging-data/ADNI/Organi...,yes,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,002_S_0295,295,002_S_0295_20090522_A1_T1_1.5T_nonaccel_Preproc,150177.0,/nas/bioint-data/neuroimaging-data/ADNI/Organi...,yes,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,002_S_0295,295,002_S_0295_20100513_A1_T1_1.5T_nonaccel_Preproc,291869.0,/nas/bioint-data/neuroimaging-data/ADNI/Organi...,yes,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,002_S_0295,295,002_S_0295_20110602_A2_T1_3T_nonaccel_Preproc,241350.0,/nas/bioint-data/neuroimaging-data/ADNI/Organi...,yes,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,002_S_0295,295,002_S_0295_20120510_A2_T1_3T_nonaccel_Preproc,308078.0,/nas/bioint-data/neuroimaging-data/ADNI/Organi...,yes,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,/nas/bioint-data/neuroimaging-data/ADNI/6DOF/0...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,002_S_0413,413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,/nas/bioint-data/neuroimaging-data/ADNI/Organi...,/nas/bioint-data/neuroimaging-data/ADNI/Organi...,/nas/bioint-data/neuroimaging-data/ADNI/Organi...,/nas/bioint-data/neuroimaging-data/ADNI/Organi...,/nas/bioint-data/neuroimaging-data/ADNI/Organi...,/nas/bioint-data/neuroimaging-data/ADNI/Organi...,/nas/bioint-data/neuroimaging-data/ADNI/Organi...,/nas/

In [19]:
import nibabel as nib
from tensorflow import keras
from tensorflow.keras import layers
import random as pyrandom
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing import image_dataset_from_directory

import nibabel as nib
from scipy import ndimage
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve, accuracy_score, f1_score, precision_score, recall_score, auc, roc_auc_score
from sklearn.utils import class_weight
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "5"

physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

/lfs1/ashaji/condawork/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Num GPUs Available:  1


In [20]:
### Hyperparameter config

init_lr = 1e-4
epochs = 50
early_stop = 100
seed = 53
drop_out = 0.5
weight_decay = 1e-4

batch_size=4
test_batch_size=4

In [21]:
def getCNN(width = 91,height = 109, depth = 91):
    
    def ann_block(inp,num,name):
        inp = tf.keras.layers.Dense(num)(inp)
        inp = tfa.layers.InstanceNormalization(center=False,scale=False)(inp)
        inp = tf.nn.relu(inp)
        return inp
    
    ##############
    
    strct = tf.keras.Input((8),name='inp2')
    inp2 = ann_block(strct,32,'ann1')
    inp2 = ann_block(inp2,16,'ann2')
    
    #LAST LAYER
    out2 = tf.keras.layers.Dense(1)(inp2)
    ##############
    
    model = keras.Model(strct,out2,name='8ANN')
    return model

model = getCNN()
model.summary()
    

2023-12-24 10:53:53.880124: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10534 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:0d:00.0, compute capability: 6.1


Model: "8ANN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inp2 (InputLayer)           [(None, 8)]               0         
                                                                 
 dense (Dense)               (None, 32)                288       
                                                                 
 instance_normalization (Ins  (None, 32)               0         
 tanceNormalization)                                             
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 instance_normalization_1 (I  (None, 16)               0         
 nstanceNormalization)                                        

In [22]:
len(t1dwistr)

458

In [23]:
strdata = df.loc[condstr,:]

In [24]:
len(strdata)

1833

In [25]:
p = (np.ceil(0.7*len(strdata))-1).astype(int)
q = (np.ceil(0.9*len(strdata))-2).astype(int)
r = len(strdata)
train = strdata[:p]
test  = strdata[p:q]
val   = strdata[q:]
len(train), len(test), len(val)

(1283, 365, 185)

In [38]:
count = 0
arr = []
for i in test['SubjID'].drop_duplicates():
    for j in train['SubjID'].drop_duplicates():
        if i == j:
            arr.append(i)
            count += 1
count

0

In [39]:
len(train), len(test), len(val)

(1283, 360, 185)

In [45]:
1283 + 360 + 185

1828

In [37]:
for i in arr:
    test.drop(test[(test['SubjID']==i) ].index,inplace = True)

/tmp/ipykernel_35870/3017501266.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(test[(test['SubjID']==i) ].index,inplace = True)


In [40]:
class DataGenerator(tf.keras.utils.Sequence):
    def rotate(self,vol):
        def scipy_rotate(vol):
            angles = [-20,-10,-5,0,5,10,20]
            angle = pyrandom.choice(angles)
            vol = ndimage.rotate(vol,angle,reshape=False)
            vol[vol<0] = 0
            vol[vol>1] = 1
            return vol
        aug_vol = tf.numpy_function(scipy_rotate,[vol],tf.float32)
        return aug_vol

    def preprocessing(self,vol):
        if(self.isTrain):
            vol1 = self.rotate(vol)
            vol1 = tf.expand_dims(vol1,axis=3)
        else:
            vol1 = tf.expand_dims(vol,axis=3)
        return vol1
    
    def read_scan(self,path):
        scan = nib.load(path)
        volume = scan.get_fdata()
        min = np.amax(volume)
        max = np.amin(volume)
        volume = (volume - min) / (max - min)
        volume = volume.astype("float32")
        return volume

    def __init__(self, data, batch_size, sample_weights=None, isTrain = True):
        self.data = data
        self.batch_size = batch_size
        self.sample_weights = sample_weights
        self.isTrain = isTrain

    def __len__(self):
        return (np.ceil(len(self.data) / float(self.batch_size))).astype(np.int)

    def __getitem__(self, idx):
        ann = self.data[['SEX','CDRSB','ADAS11','ADAS13','MMSE','MOCA','APOE A1','APOE A2']].values.tolist()
        labels = self.data['AGE_at_scan'].astype(np.float32)
        
        batch_ann = ann[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_y = labels[idx * self.batch_size: (idx + 1) * self.batch_size]
     
        str_data = np.asarray(batch_ann)
        
        return (str_data, np.array(batch_y))
    
    def on_epoch_end(self):
        if(self.isTrain):
            self.data = self.data.sample(frac=1)

In [42]:
dg_train = DataGenerator(train,batch_size)
dg_val = DataGenerator(val,batch_size,isTrain=False)
dg_test = DataGenerator(test,batch_size,isTrain=False)

In [43]:
model.compile(
    loss='mse',
    optimizer = keras.optimizers.AdamW(learning_rate=1e-2,weight_decay=1e-5),
    metrics = [tf.keras.metrics.RootMeanSquaredError(name='rmse'),tf.keras.metrics.MeanAbsoluteError(name='mae')]   
)
early = keras.callbacks.EarlyStopping(monitor='val_mae',patience = 100, verbose = 1, restore_best_weights = True)

history = model.fit(
    dg_train,
    validation_data = dg_val,
    epochs = 2000,
    verbose = 1,
    callbacks = [early]
)
model.save('/lfs1/ashaji/Imputation_Problem/EMBC/STR/modelonlySTR')

Epoch 1/2000


/tmp/ipykernel_35870/3680906809.py:37: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.data) / float(self.batch_size))).astype(np.int)
/tmp/ipykernel_35870/3680906809.py:37: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additiona

309/321 [===========================>..] - ETA: 0s - loss: 5402.9907 - rmse: 73.5050 - mae: 73.1721

/tmp/ipykernel_35870/3680906809.py:37: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.data) / float(self.batch_size))).astype(np.int)
/tmp/ipykernel_35870/3680906809.py:37: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additiona

321/321 [==============================] - 5s 6ms/step - loss: 5391.1401 - rmse: 73.4244 - mae: 73.0898 - val_loss: 4990.8721 - val_rmse: 70.6461 - val_mae: 70.3735
Epoch 2/2000
 36/321 [==>...........................] - ETA: 1s - loss: 5035.6738 - rmse: 70.9625 - mae: 70.6216

...

321/321 [==============================] - 2s 5ms/step - loss: 48.0042 - rmse: 6.9285 - mae: 5.5876 - val_loss: 39.6454 - val_rmse: 6.2965 - val_mae: 5.2426
Epoch 132/2000


/tmp/ipykernel_35870/3680906809.py:37: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.data) / float(self.batch_size))).astype(np.int)
/tmp/ipykernel_35870/3680906809.py:37: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additiona

319/321 [============================>.] - ETA: 0s - loss: 47.8001 - rmse: 6.9138 - mae: 5.5821Restoring model weights from the end of the best epoch: 32.


/tmp/ipykernel_35870/3680906809.py:37: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.data) / float(self.batch_size))).astype(np.int)


321/321 [==============================] - 2s 6ms/step - loss: 48.0114 - rmse: 6.9290 - mae: 5.5904 - val_loss: 39.5337 - val_rmse: 6.2876 - val_mae: 5.2340
Epoch 132: early stopping
INFO:tensorflow:Assets written to: /lfs1/ashaji/Imputation_Problem/EMBC/STR/modelonlySTR/assets


In [44]:
model.evaluate(dg_test)

29/90 [========>.....................] - ETA: 0s - loss: 53.0278 - rmse: 7.2820 - mae: 5.9276  

/tmp/ipykernel_35870/3680906809.py:37: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.data) / float(self.batch_size))).astype(np.int)
2023-12-24 13:14:16.377480: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


90/90 [==============================] - 0s 4ms/step - loss: 46.3950 - rmse: 6.8114 - mae: 5.6834


[46.394954681396484, 6.811384201049805, 5.683448314666748]

In [54]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    1e-2,decay_steps=10,decay_rate=0.96,staircase=True
)
model.compile(
    loss='mse',
    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics = [tf.keras.metrics.RootMeanSquaredError(name='rmse'),tf.keras.metrics.MeanAbsoluteError(name='mae')]   
)
filepath = '/lfs1/ashaji/Imputation_Problem/further/STR3/modelonlySTR-epoch{epoch:02d}-val_loss{val_loss:.2f}'
check = keras.callbacks.ModelCheckpoint(filepath,monitor='val_mae',verbose=1,save_best_only=True,mode='auto')
early = keras.callbacks.EarlyStopping(monitor='val_mae',patience = 20, verbose = 1, restore_best_weights = True)

history = model.fit(
    dg_train,
    validation_data = dg_val,
    epochs = 200,
    verbose = 1,
    callbacks = [check,early]
)

Epoch 1/200


/tmp/ipykernel_31192/2520707933.py:33: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.data) / float(self.batch_size))).astype(np.int)
/tmp/ipykernel_31192/2520707933.py:33: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additiona

332/339 [============================>.] - ETA: 0s - loss: 47.8629 - rmse: 6.9183 - mae: 5.5867

/tmp/ipykernel_31192/2520707933.py:33: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.data) / float(self.batch_size))).astype(np.int)



Epoch 1: val_mae improved from inf to 5.46553, saving model to /lfs1/ashaji/Imputation_Problem/further/STR3/modelonlySTR-epoch01-val_loss41.95
INFO:tensorflow:Assets written to: /lfs1/ashaji/Imputation_Problem/further/STR3/modelonlySTR-epoch01-val_loss41.95/assets
339/339 [==============================] - 5s 10ms/step - loss: 47.7633 - rmse: 6.9111 - mae: 5.5750 - val_loss: 41.9533 - val_rmse: 6.4771 - val_mae: 5.4655
Epoch 2/200
 21/339 [>.............................] - ETA: 1s - loss: 50.6077 - rmse: 7.1139 - mae: 5.5758

...

/tmp/ipykernel_31192/2520707933.py:33: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.data) / float(self.batch_size))).astype(np.int)



Epoch 9: val_mae did not improve from 5.45463
339/339 [==============================] - 2s 6ms/step - loss: 47.5758 - rmse: 6.8975 - mae: 5.5718 - val_loss: 41.8240 - val_rmse: 6.4671 - val_mae: 5.4559
Epoch 10/200
 33/339 [=>............................] - ETA: 1s - loss: 43.8120 - rmse: 6.6191 - mae: 5.2784

/tmp/ipykernel_31192/2520707933.py:33: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.data) / float(self.batch_size))).astype(np.int)


283/339 [========================>.....] - ETA: 0s - loss: 48.1436 - rmse: 6.9386 - mae: 5.5980

KeyboardInterrupt: 

In [51]:
dg_testall = DataGenerator(testall,batch_size)

In [70]:
model = keras.models.load_model('/lfs1/ashaji/Imputation_Problem/further/STR2/modelonlySTR-epoch52-val_loss42.21')

In [71]:
model.evaluate(dg_test)

/tmp/ipykernel_14261/2520707933.py:33: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.data) / float(self.batch_size))).astype(np.int)
2023-10-11 18:22:49.082261: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


85/85 [==============================] - 1s 4ms/step - loss: 38.0020 - rmse: 6.1646 - mae: 4.9562


[38.00200653076172, 6.164576530456543, 4.9561872482299805]

In [72]:
model.evaluate(dg_testall)

23/23 [==============================] - 0s 4ms/step - loss: 53.1265 - rmse: 7.2888 - mae: 6.1786


/tmp/ipykernel_14261/2520707933.py:33: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.data) / float(self.batch_size))).astype(np.int)
2023-10-11 18:22:52.235042: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


[53.12651824951172, 7.288794040679932, 6.178591728210449]

In [ ]:
model.evaluate(dg_val_MCI)

In [ ]:
model.evaluate(dg_val_Dmt)

In [ ]:
len(strDmt)

In [ ]:
len(strMCI)

In [57]:
modelDWI = keras.models.load_model('/lfs1/ashaji/Imputation_Problem/saved_models/model_only_DWI_FA-epoch36-val_loss32.66')
